<a href="https://colab.research.google.com/github/sanjaykhanssk/buticku/blob/master/Tikku_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# USE /ENABLE   GPU   IN COLAB

In [0]:

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp 'drive/My Drive/projects/DrQA/DrQAtrained.zip' 'drqa.zip'

In [0]:
!unzip drqa.zip

In [0]:
%cd DrQA

/content/DrQA


In [0]:
!pip install -r requirements.txt

In [0]:
!python3 -m spacy download en


In [0]:
!pip install python-firebase

In [0]:
# !python interact.py

In [0]:
import time
import argparse
import torch
import msgpack
from drqa.model import DocReaderModel
from drqa.utils import str2bool
from prepro import annotate, to_id, init
from train import BatchGen



model = 'models/best_model.pt'
cuda = torch.cuda.is_available()

if cuda:
    checkpoint = torch.load(model)
else:
    checkpoint = torch.load(model, map_location=lambda storage, loc: storage)

state_dict = checkpoint['state_dict']
opt = checkpoint['config']
with open('SQuAD/meta.msgpack', 'rb') as f:
    meta = msgpack.load(f, encoding='utf8')
    
embedding = torch.Tensor(meta['embedding'])



opt['pretrained_words'] = True
opt['vocab_size'] = embedding.size(0)
opt['embedding_dim'] = embedding.size(1)
opt['pos_size'] = len(meta['vocab_tag'])
opt['ner_size'] = len(meta['vocab_ent'])
opt['cuda'] = cuda
BatchGen.pos_size = opt['pos_size']
BatchGen.ner_size = opt['ner_size']
model = DocReaderModel(opt, embedding, state_dict)
w2id = {w: i for i, w in enumerate(meta['vocab'])}
tag2id = {w: i for i, w in enumerate(meta['vocab_tag'])}
ent2id = {w: i for i, w in enumerate(meta['vocab_ent'])}
init()

def get_answer(evidence , question):
    if question == 'question_read':
        pass
    print(question)
    id_ = 0
    # try:
    #     while True:
    #         evidence = input('Evidence: ')
    #         if evidence.strip():
    #             break
    #     while True:
    #         question = input('Question: ')
    #         if question.strip():
    #             break
    # except EOFError:
    #     print()
#         break
    id_ += 1
#     start_time = time.time()
    annotated = annotate(('interact-{}'.format(id_), evidence, question), meta['wv_cased'])
    model_in = to_id(annotated, w2id, tag2id, ent2id)
    model_in = next(iter(BatchGen([model_in], batch_size=1, gpu=cuda, evaluation=True)))
    prediction = model.predict(model_in)[0]
#     end_time = time.time()
#     print('Answer: {}'.format(prediction))
#     print('Time: {:.4f}s'.format(end_time - start_time))
    
    return prediction

from firebase import firebase
firebaseref = firebase.FirebaseApplication('https://buticku16.firebaseio.com/')
    
    
while 1:
    question = firebaseref.get('/question',None)

    if question == 'question_read':
        continue
    if question == 'break':
        break
    else:
        content = firebaseref.get('/text',None)
        firebaseref.put('/','answer',get_answer(content , question))
        firebaseref.put('/','question','question_read')

can i buy this one?
can i buy this one?
tell me about camera
what about camera
is this positive?
is this awsome?
can i drop it from mars??
ok good going to buy this product
thank you
is this worthy product?
tell me more about camera?
what about camera
thank you
